## Compare metadetect catalog types

This is a simple script to measure the match efficiency between different metadectect shear catalogs

#### Standard imports

In [ ]:
import hpmcm
import glob
import os
import numpy as np

#### Set up the configuration

In [ ]:
DATADIR = "test_data"   # Input data directory
shear_st = "0p01"       # Applied shear as a string
shear = 0.01            # Decimal version of applied shear
shear_type = "wmom"     # which object characterization to use 
tract = 10463           # which tract to study

SOURCE_TABLEFILES = sorted(glob.glob(os.path.join(DATADIR, f"shear_{shear_type}_{shear_st}_uncleaned_{tract}_*.pq")))
SOURCE_TABLEFILES.reverse()
VISIT_IDS = np.arange(len(SOURCE_TABLEFILES))

PIXEL_R2CUT = 4.         # Cut at distance**2 = 4 pixels
PIXEL_MATCH_SCALE = 1    # Use pixel scale to do matching

#### Make the matcher, reduce the data

In [ ]:
matcher = hpmcm.ShearMatch.createShearMatch(pixelR2Cut=PIXEL_R2CUT, pixelMatchScale=PIXEL_MATCH_SCALE)
matcher.reduceData(SOURCE_TABLEFILES, VISIT_IDS)

#### Run the data

Note the option to run all the cells.  By default we only run a small subset for testing

In [ ]:
do_partial = True
if do_partial:
    xRange = range(50, 70)
    yRange = range(170, 190)
    #xRange = [55]
    #yRange = [170]
    matcher.analysisLoop(xRange, yRange)
else:
    matcher.analysisLoop()

#### Classify the objects by match type

This looks at the characteristics of the matched objects and categorizes them.

In [ ]:
objLists = hpmcm.classify.classifyObjects(matcher)
hpmcm.classify.printObjectTypes(objLists)

In [ ]:
n_good = len(objLists['ideal'])
bad_list = ['edge_mixed', 'edge_missing', 'edge_extra', 'orphan', 'missing', 'two_missing', 'many_missing', 'extra', 'caught']
n_bad = np.sum([len(objLists[x]) for x in bad_list])

In [ ]:
n_good/(n_good+n_bad)